# Models: word2vec - Evaluation

In [1]:
import numpy as np
import os
import pandas as pd
import re
import time

from gensim.models import word2vec

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

from pprint import pprint

## Set input data folder

In [2]:
data_in = '../data/03_preprocessed_01_non_letters_to_empty_space'

pprint(sorted(os.listdir(data_in)))

['clck.csv', 'news.csv']


In [3]:
data_out = '../data/04_models_word2vec'
# pprint(sorted(os.listdir(data_out)))

## Read data into data frames

In [4]:
clck = pd.read_csv(f'{data_in}/clck.csv', header=None)
clck.columns = ['_headlines']
clck['clickbait'] = pd.Series(np.ones(len(clck)) == 1)
clck[0:len(clck):5000]

,_headlines,clickbait
0,<number> Indian <number> Indian Unrelat...,True
5000,<number> Characters From Your Childhood That ...,True
10000,<number> Pictures That Are Way Too Real For P...,True
15000,<number> Gifts I Actually Really Want,True
20000,According To Bill Gates Something Deadly Is C...,True
25000,Couple Adopts <number> Kids When The Neighbo...,True
30000,Housing crisis the empty nester s guide to do...,True
35000,Lily Allen Shares A Video Of Herself Dancing T...,True
40000,Should Hollywood be afraid of the march of Net...,True
45000,These Beautiful Pieces Of Trans Artwork Offer ...,True


In [5]:
news = pd.read_csv(f'{data_in}/news.csv', header=None)
news.columns = ['_headlines']
news['clickbait'] = pd.Series(np.ones(len(news)) == 0)
news[0:len(news):5000]

,_headlines,clickbait
0,I Wish I Had Gone Out With Him Last Night ...,False
5000,American musician Bo Diddley dies at age <num...,False
10000,CHICAGO AP Illinois new medical marijuana...,False
15000,Developing Nations May Be Driving World Economy,False
20000,Four young girls killed in Zambia wall collapse,False
25000,In Uneasy Time Saudi Prince Provides a Hope o...,False
30000,Louisiana Senate Race Foster Campbell Is Demo...,False
35000,New Virginia law bans smoking in cars with chi...,False
40000,Priest arrested on <number> counts of child p...,False
45000,Someone Has Made The Trump Sandwich A Reality ...,False


### Merge data frames

In [6]:
df = pd.concat([clck, news], ignore_index=True)
df[0:len(df):5000]

,_headlines,clickbait
0,<number> Indian <number> Indian Unrelat...,True
5000,<number> Characters From Your Childhood That ...,True
10000,<number> Pictures That Are Way Too Real For P...,True
15000,<number> Gifts I Actually Really Want,True
20000,According To Bill Gates Something Deadly Is C...,True
25000,Couple Adopts <number> Kids When The Neighbo...,True
30000,Housing crisis the empty nester s guide to do...,True
35000,Lily Allen Shares A Video Of Herself Dancing T...,True
40000,Should Hollywood be afraid of the march of Net...,True
45000,These Beautiful Pieces Of Trans Artwork Offer ...,True


## Preprocessing

In [7]:
if os.path.exists(f'{data_out}/../data.csv'):
    df = pd.read_csv(f'{data_out}/../data.csv', index_col='Unnamed: 0')
    df = df.fillna('<nan>')
    
else:
    for regex in [None]:
        for stop_words in [None, set(stopwords.words('english'))]:
            for lemmatizer in [None, WordNetLemmatizer()]:
                for stemmer in [None, PorterStemmer()]:

                    settings  = ''
                    settings += '' if regex is None else '_re'
                    settings += '' if stop_words is None else '_sw'
                    settings += '' if lemmatizer is None else '_lm'
                    settings += '' if stemmer is None else '_sm'

                    print(f'headlines{settings:20s} ...', end=' ')

                    t = time.time()
                    headlines = list()

                    for i in range(len(df)):
                        # if (i+1) % 1000 == 0:
                        # print(f'Review {i+1:>6d} of {len(df):>6d}')

                        text = df['_headlines'][i]

                        # split the headline into words
                        text = re.split('\s+', text.lower().strip())

                        # replace unnecessary patterns
                        if regex is not None:
                            text = regex.sub(' ', text)

                        # remove stop-words
                        if stop_words is not None:
                            text = [w for w in text if w not in stop_words]

                        # lemmatization
                        if lemmatizer is not None:
                            text = [lemmatizer.lemmatize(w) for w in text]

                        # stemming
                        if stemmer is not None:
                            text = [stemmer.stem(w) for w in text]

                        headlines.append(' '.join(text))

                    df[f'headlines{settings}'] = pd.Series(headlines)

                    print(f'Done in {int(np.ceil(time.time() - t))} seconds!')

    df.to_csv(f'{data_out}/../data.csv')

## 10-fold cross validation

In [8]:
kf = KFold(n_splits=10, shuffle=True, random_state=24101989)

## Classification

### Functions

In [9]:
def init_dfs():
    acc_df = pd.DataFrame(index=[i+1 for i in range(kf.n_splits)])
    out_df = pd.DataFrame(columns=['i'], index=df.index)
    tim_df = pd.DataFrame(index=[i+1 for i in range(kf.n_splits)])

    i = 1
    for train_index, test_index in kf.split(df):
        out_df['i'][test_index] = i
        i += 1
        
    return acc_df, out_df, tim_df

In [10]:
def init_series():
    acc_s = pd.Series(data=[np.nan for _ in range(kf.n_splits)],
                      index=acc_df.index)
    out_s = pd.Series(data=[np.nan for _ in range(len(df))],
                      index=out_df.index)
    tim_s = pd.Series(data=[np.nan for _ in range(kf.n_splits)],
                      index=tim_df.index)

    return acc_s, out_s, tim_s

In [11]:
def fit_and_predict(cl, cl_name, data_vectors):
    acc_s, out_s, tim_s = init_series()
    
    i = 1
    for train_index, test_index in kf.split(df):
        t = time.time()

        train_x = df[f'{s}'][train_index]
        train_y = df['clickbait'][train_index]
        test_x  = df[f'{s}'][test_index]
        test_y  = df['clickbait'][test_index]

        train_dv = data_vectors[train_index]
        test_dv  = data_vectors[test_index]

        # fit & predict
        cl = cl.fit(train_dv, train_y)
        results = pd.Series(cl.predict(test_dv),
                            index=test_y.index)

        # save results of the i-th split
        acc_s[i]            = np.sum(results == test_y) / len(test_y)
        out_s[test_y.index] = results
        tim_s[i]            = np.around(time.time() - t, decimals=3)
        print(f'{cl_name:30s} [{i:>2d}] | {(acc_s[i] * 100):>6.3f} % | {int(np.ceil(tim_s[i])):>4d} s')
        i += 1

    return acc_s, out_s, tim_s

In [12]:
def run(cl, cl_name, data_vectors):
    acc_s, out_s, tim_s = fit_and_predict(cl, cl_name, data_vectors)
    acc_df[cl_name] = acc_s
    out_df[cl_name] = out_s
    tim_df[cl_name] = tim_s
    
    print()
    return

#### Vector Averaging

In [13]:
def make_feature_vector(words, model, ft):
    feature_vector = np.zeros((ft,), dtype='float32')

    n_words = 0.
    index2word_set = set(model.wv.index2word)

    for word in words:
        if word in index2word_set: 
            n_words = n_words + 1.
            feature_vector = np.add(feature_vector, model[word])

    feature_vector = np.divide(feature_vector, n_words)
    return feature_vector


def get_average_feature_vectors(reviews, model, ft):
    review_feature_vectors = np.zeros((len(reviews), ft), dtype='float32')

    counter = 0
    for review in reviews:
#         if counter % 1000 == 0:
#             print(f'Review {counter} of {len(reviews)}')
        review_feature_vectors[counter] = make_feature_vector(review, model, ft)
        counter = counter + 1
    return review_feature_vectors

### Fit & Predict

#### word2vec

##### Model parameters

In [15]:
n_features = [500]            # Word vector dimensionality | 100, 500; 1000, 2500, 5000
min_word_count = [5, 10, 25]  # Minimum word count
n_workers = 8                 # Number of threads to run in parallel
context = [3, 5, 7]           # Context-window size
downsampling = 1e-3           # Downsample setting for frequent words

##### Initial Training

In [16]:
# settings of interest
soi = [
       # 'headlines',
       # 'headlines_sw',
       # 'headlines_lm',
       'headlines_sm',
       # 'headlines_sw_lm',
       # 'headlines_sw_sm',
       # 'headlines_lm_sm',
       # 'headlines_sw_lm_sm',
      ]

model_time_dict = dict()
now = ''.join([f'{item:02d}' for item in time.localtime()[:6]])

for s in soi:
    setting = s[9:]
    
    # Prepare word2vec input
    sentences = [headline.split(' ') for headline in list(df[f'{s}'])]
    
    w2v_input = []
    for sentence in sentences:
        w2v_input.append([word for word in sentence if len(word) > 1])

    for ft in [100]:  # n_features
        for mw in [10]:  # min_word_count
            for cx in [5]:  # context
                cycle_time = time.time()
                
                # Initialize dataframes
                acc_df, out_df, tim_df = init_dfs()
                
                # Create word2vec model
                model_name = f'{ft}ft_{mw}mw_{cx}cx{setting}'
                model_load = model_name in os.listdir(f'{data_out}/models')
                
                if model_load:
                    model = word2vec.Word2Vec.load(f'{data_out}/models/{model_name}')
                else:
                    model_time_dict[model_name] = time.time()

                    print(f'\nTraining {model_name}...', end=' ')
                    t = time.time()
                    model = word2vec.Word2Vec(w2v_input, workers=n_workers,
                                              size=ft, min_count=mw,
                                              window=cx, sample=downsampling)
                    model.init_sims(replace=True)
                    model.save(f'{data_out}/models/{model_name}')
                    print(f'Done in {(time.time() - t):.2f} seconds!')

                # Create data vectors
                print('Creating average feature vectors...', end=' ')
                t = time.time()
                data_vectors = get_average_feature_vectors(sentences, model, ft)
                data_vectors = np.nan_to_num(data_vectors)
                # np.save(f'{data_out}/data_vectors/{model_name}', data_vectors)
                print(f'Done in {(time.time() - t):.2f} seconds!')
                
                if not model_load:
                    model_time_dict[model_name] = np.abs(time.time() - 
                                                         model_time_dict[model_name])
                    pd.DataFrame(model_time_dict, index=['Time']).transpose() \
                      .to_csv(f'{data_out}/{ft}ft_{now}_model_time.csv')
    
                # Train & test
                # - Extra Trees
                for est in [100]:  # default = 10
                    run(ExtraTreesClassifier(n_estimators=est, n_jobs=-1,
                                             verbose=True, bootstrap=True),
                        f'{model_name}_ET_{est}est', data_vectors)

                # - Logistic Regression
                for c in [1.00]:
                    run(LogisticRegression(C=c, max_iter=1000,
                                           n_jobs=-1, verbose=True),
                        f'{model_name}_LogReg_{c:.2f}C', data_vectors)

                # - Naïve Bayes
                for alpha in [1.00]:
                    run(BernoulliNB(alpha=alpha), 
                        f'{model_name}_NB_{alpha:.2f}alpha', data_vectors)

                # - Neural Network
                for max_iter in [1000]:
                    run(MLPClassifier(max_iter=max_iter, verbose=True),
                        f'{model_name}_NN_{max_iter}iter', data_vectors)

                # - Random Forests
                for est in [100]:  # default = 10
                    run(RandomForestClassifier(n_estimators=est, n_jobs=-1,
                                               verbose=True, bootstrap=True),
                        f'{model_name}_RF_{est}e', data_vectors)

                print(f'\n========== {model_name} cycle finished in {int(np.ceil(time.time() - cycle_time))} seconds. ==========\n\n\n')

                # Save model dataframes
                acc_df = acc_df.transpose()
                tim_df = tim_df.transpose()

                acc_df.to_csv(f'{data_out}/{model_name}_{now}_acc.csv')
                out_df.to_csv(f'{data_out}/{model_name}_{now}_out.csv')
                tim_df.to_csv(f'{data_out}/{model_name}_{now}_tim.csv')

Creating average feature vectors... 

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


Done in 48.93 seconds!
Iteration 1, loss = 0.45156193
Iteration 2, loss = 0.35701665
Iteration 3, loss = 0.35362293
Iteration 4, loss = 0.35177234
Iteration 5, loss = 0.35046596
Iteration 6, loss = 0.34915801
Iteration 7, loss = 0.34806799
Iteration 8, loss = 0.34694963
Iteration 9, loss = 0.34558745
Iteration 10, loss = 0.34431827
Iteration 11, loss = 0.34287248
Iteration 12, loss = 0.34177714
Iteration 13, loss = 0.34078799
Iteration 14, loss = 0.33999503
Iteration 15, loss = 0.33916011
Iteration 16, loss = 0.33860722
Iteration 17, loss = 0.33813154
Iteration 18, loss = 0.33776646
Iteration 19, loss = 0.33743718
Iteration 20, loss = 0.33718154
Iteration 21, loss = 0.33683930
Iteration 22, loss = 0.33650721
Iteration 23, loss = 0.33624096
Iteration 24, loss = 0.33592348
Iteration 25, loss = 0.33589395
Iteration 26, loss = 0.33573907
Iteration 27, loss = 0.33558695
Iteration 28, loss = 0.33513817
Iteration 29, loss = 0.33513090
Iteration 30, loss = 0.33483620
Iteration 31, loss = 0.334

Iteration 30, loss = 0.33364716
Iteration 31, loss = 0.33344206
Iteration 32, loss = 0.33320473
Iteration 33, loss = 0.33321087
Iteration 34, loss = 0.33276440
Iteration 35, loss = 0.33293821
Iteration 36, loss = 0.33274912
Iteration 37, loss = 0.33241259
Iteration 38, loss = 0.33236137
Iteration 39, loss = 0.33236151
Iteration 40, loss = 0.33217777
Iteration 41, loss = 0.33201409
Iteration 42, loss = 0.33171874
Iteration 43, loss = 0.33159587
Iteration 44, loss = 0.33153589
Iteration 45, loss = 0.33142328
Iteration 46, loss = 0.33148228
Iteration 47, loss = 0.33140096
Iteration 48, loss = 0.33126905
Iteration 49, loss = 0.33089288
Iteration 50, loss = 0.33089860
Iteration 51, loss = 0.33080551
Iteration 52, loss = 0.33066314
Iteration 53, loss = 0.33057230
Iteration 54, loss = 0.33051816
Iteration 55, loss = 0.33036146
Iteration 56, loss = 0.33028444
Iteration 57, loss = 0.33032823
Iteration 58, loss = 0.33009124
Iteration 59, loss = 0.32995585
Iteration 60, loss = 0.32992263
Iteratio

Iteration 121, loss = 0.31914481
Iteration 122, loss = 0.31904361
Iteration 123, loss = 0.31892922
Iteration 124, loss = 0.31874002
Iteration 125, loss = 0.31886229
Iteration 126, loss = 0.31868389
Iteration 127, loss = 0.31838475
Iteration 128, loss = 0.31824801
Iteration 129, loss = 0.31830077
Iteration 130, loss = 0.31805520
Iteration 131, loss = 0.31789417
Iteration 132, loss = 0.31795366
Iteration 133, loss = 0.31761064
Iteration 134, loss = 0.31777525
Iteration 135, loss = 0.31743182
Iteration 136, loss = 0.31731562
Iteration 137, loss = 0.31754506
Iteration 138, loss = 0.31715494
Iteration 139, loss = 0.31701604
Iteration 140, loss = 0.31696742
Iteration 141, loss = 0.31691778
Iteration 142, loss = 0.31663614
Iteration 143, loss = 0.31636704
Iteration 144, loss = 0.31660111
Iteration 145, loss = 0.31636360
Iteration 146, loss = 0.31618425
Iteration 147, loss = 0.31609970
Iteration 148, loss = 0.31605095
Iteration 149, loss = 0.31615570
Training loss did not improve more than tol

...

Iteration 4, loss = 0.32686018
Iteration 5, loss = 0.32380312
Iteration 6, loss = 0.32135167
Iteration 7, loss = 0.31929012
Iteration 8, loss = 0.31776977
Iteration 9, loss = 0.31640082
Iteration 10, loss = 0.31505255
Iteration 11, loss = 0.31418938
Iteration 12, loss = 0.31301485
Iteration 13, loss = 0.31192795
Iteration 14, loss = 0.31108391
Iteration 15, loss = 0.31026172
Iteration 16, loss = 0.30930577
Iteration 17, loss = 0.30837359
Iteration 18, loss = 0.30769654
Iteration 19, loss = 0.30735679
Iteration 20, loss = 0.30652781
Iteration 21, loss = 0.30585064
Iteration 22, loss = 0.30541696
Iteration 23, loss = 0.30468057
Iteration 24, loss = 0.30440909
Iteration 25, loss = 0.30385237
Iteration 26, loss = 0.30326209
Iteration 27, loss = 0.30305570
Iteration 28, loss = 0.30274703
Iteration 29, loss = 0.30186659
Iteration 30, loss = 0.30130178
Iteration 31, loss = 0.30127980
Iteration 32, loss = 0.30058886
Iteration 33, loss = 0.30042610
Iteration 34, loss = 0.29991416
Iteration 35, 

Iteration 11, loss = 0.31461378
Iteration 12, loss = 0.31388116
Iteration 13, loss = 0.31259574
Iteration 14, loss = 0.31163716
Iteration 15, loss = 0.31099295
Iteration 16, loss = 0.31009893
Iteration 17, loss = 0.30953503
Iteration 18, loss = 0.30889496
Iteration 19, loss = 0.30818061
Iteration 20, loss = 0.30737227
Iteration 21, loss = 0.30689407
Iteration 22, loss = 0.30645054
Iteration 23, loss = 0.30591376
Iteration 24, loss = 0.30540176
Iteration 25, loss = 0.30490384
Iteration 26, loss = 0.30439915
Iteration 27, loss = 0.30436437
Iteration 28, loss = 0.30379046
Iteration 29, loss = 0.30338572
Iteration 30, loss = 0.30322348
Iteration 31, loss = 0.30252770
Iteration 32, loss = 0.30200493
Iteration 33, loss = 0.30165453
Iteration 34, loss = 0.30131892
Iteration 35, loss = 0.30124615
Iteration 36, loss = 0.30064187
Iteration 37, loss = 0.30050367
Iteration 38, loss = 0.30031407
Iteration 39, loss = 0.29974724
Iteration 40, loss = 0.29949135
Iteration 41, loss = 0.29927645
Iteratio

Iteration 131, loss = 0.28234292
Iteration 132, loss = 0.28227600
Iteration 133, loss = 0.28194930
Iteration 134, loss = 0.28202393
Iteration 135, loss = 0.28211511
Iteration 136, loss = 0.28189162
Iteration 137, loss = 0.28157392
Iteration 138, loss = 0.28194283
Iteration 139, loss = 0.28138719
Iteration 140, loss = 0.28136084
Iteration 141, loss = 0.28129754
Iteration 142, loss = 0.28154922
Iteration 143, loss = 0.28081411
Iteration 144, loss = 0.28087181
Iteration 145, loss = 0.28102455
Iteration 146, loss = 0.28031509
Iteration 147, loss = 0.28062127
Iteration 148, loss = 0.28082564
Iteration 149, loss = 0.28094795
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.
100ft_10mw_5cx_lm_NN_1000iter_reluactiv_1.0pt_1e-05tl [ 6] | 87.995 % |  322 s
Iteration 1, loss = 0.38013708
Iteration 2, loss = 0.33803292
Iteration 3, loss = 0.33108068
Iteration 4, loss = 0.32718753
Iteration 5, loss = 0.32440224
Iteration 6, loss = 0.32249474
Iteration 7, loss

Iteration 13, loss = 0.31175610
Iteration 14, loss = 0.31106886
Iteration 15, loss = 0.30999456
Iteration 16, loss = 0.30890774
Iteration 17, loss = 0.30827728
Iteration 18, loss = 0.30748989
Iteration 19, loss = 0.30667399
Iteration 20, loss = 0.30627592
Iteration 21, loss = 0.30579680
Iteration 22, loss = 0.30523755
Iteration 23, loss = 0.30433304
Iteration 24, loss = 0.30394728
Iteration 25, loss = 0.30348415
Iteration 26, loss = 0.30292558
Iteration 27, loss = 0.30256055
Iteration 28, loss = 0.30208432
Iteration 29, loss = 0.30152335
Iteration 30, loss = 0.30116645
Iteration 31, loss = 0.30062891
Iteration 32, loss = 0.30032171
Iteration 33, loss = 0.29987969
Iteration 34, loss = 0.29962762
Iteration 35, loss = 0.29937588
Iteration 36, loss = 0.29872464
Iteration 37, loss = 0.29848471
Iteration 38, loss = 0.29821821
Iteration 39, loss = 0.29774310
Iteration 40, loss = 0.29768323
Iteration 41, loss = 0.29723786
Iteration 42, loss = 0.29686490
Iteration 43, loss = 0.29664314
Iteratio

Iteration 125, loss = 0.28183325
Iteration 126, loss = 0.28184627
Iteration 127, loss = 0.28167483
Iteration 128, loss = 0.28169133
Iteration 129, loss = 0.28136831
Iteration 130, loss = 0.28151177
Iteration 131, loss = 0.28120129
Iteration 132, loss = 0.28119302
Iteration 133, loss = 0.28094166
Iteration 134, loss = 0.28111400
Iteration 135, loss = 0.28060969
Iteration 136, loss = 0.28070659
Iteration 137, loss = 0.28067756
Iteration 138, loss = 0.28056488
Iteration 139, loss = 0.28037222
Iteration 140, loss = 0.28007174
Iteration 141, loss = 0.28042036
Iteration 142, loss = 0.28009129
Iteration 143, loss = 0.27983907
Iteration 144, loss = 0.27972186
Iteration 145, loss = 0.27957262
Iteration 146, loss = 0.27989853
Iteration 147, loss = 0.27947430
Iteration 148, loss = 0.27926203
Iteration 149, loss = 0.27919222
Iteration 150, loss = 0.27922517
Iteration 151, loss = 0.27939416
Iteration 152, loss = 0.27946240
Training loss did not improve more than tol=0.000010 for two consecutive epo

##### Additional training

In [16]:
# settings of interest
soi = [
       # 'headlines',
       # 'headlines_sw',
       'headlines_lm',
       # 'headlines_sm',
       # 'headlines_sw_lm',
       # 'headlines_sw_sm',
       # 'headlines_lm_sm',
       # 'headlines_sw_lm_sm',
      ]

model_time_dict = dict()
now = ''.join([f'{item:02d}' for item in time.localtime()[:6]])

for s in soi:
    setting = s[9:]
    
    # Prepare word2vec input
    sentences = [headline.split(' ') for headline in list(df[f'{s}'])]
    
    w2v_input = []
    for sentence in sentences:
        w2v_input.append([word for word in sentence if len(word) > 1])

    for ft in [100]:  # n_features
        for mw in [10]:  # min_word_count
            for cx in [5]:  # context
                cycle_time = time.time()
                
                # Initialize dataframes
                acc_df, out_df, tim_df = init_dfs()
                
                # Create word2vec model
                model_name = f'{ft}ft_{mw}mw_{cx}cx{setting}'
                model_load = model_name in os.listdir(f'{data_out}/models')
                
                if model_load:
                    model = word2vec.Word2Vec.load(f'{data_out}/models/{model_name}')
                else:
                    model_time_dict[model_name] = time.time()

                    print(f'\nTraining {model_name}...', end=' ')
                    t = time.time()
                    model = word2vec.Word2Vec(w2v_input, workers=n_workers,
                                              size=ft, min_count=mw,
                                              window=cx, sample=downsampling)
                    model.init_sims(replace=True)
                    model.save(f'{data_out}/models/{model_name}')
                    print(f'Done in {(time.time() - t):.2f} seconds!')

                # Create data vectors
                print('Creating average feature vectors...', end=' ')
                t = time.time()
                data_vectors = get_average_feature_vectors(sentences, model, ft)
                data_vectors = np.nan_to_num(data_vectors)
                # np.save(f'{data_out}/data_vectors/{model_name}', data_vectors)
                print(f'Done in {(time.time() - t):.2f} seconds!')
                
                if not model_load:
                    model_time_dict[model_name] = np.abs(time.time() - 
                                                         model_time_dict[model_name])
                    pd.DataFrame(model_time_dict, index=['Time']).transpose() \
                      .to_csv(f'{data_out}/{ft}ft_{now}_model_time.csv')
    
                # Train & test
                # - Extra Trees
#                 for est in [100]:  # default = 10
#                     run(ExtraTreesClassifier(n_estimators=est, n_jobs=-1,
#                                              verbose=True, bootstrap=True),
#                         f'{model_name}_ET_{est}est', data_vectors)

                # - Logistic Regression
#                 for c in [1.00]:
#                     run(LogisticRegression(C=c, max_iter=1000,
#                                            n_jobs=-1, verbose=True),
#                         f'{model_name}_LogReg_{c:.2f}C', data_vectors)

                # - Naïve Bayes
#                 for alpha in [1.00]:
#                     run(BernoulliNB(alpha=alpha), 
#                         f'{model_name}_NB_{alpha:.2f}alpha', data_vectors)

                # - Neural Network
                for max_iter in [1000]:
                    for activ in ['logistic', 'relu']:
                        for pt in [0.25, 0.50, 0.75, 1.00]:
                            for tl in [1e-4, 1e-5]:
                                run(MLPClassifier(max_iter=max_iter, verbose=True,
                                                  activation=activ,
                                                  power_t=pt,
                                                  tol=tl),
                                f'{model_name}_NN_{max_iter}iter_{activ}_{pt}pt_{tl}tl', data_vectors)

                # - Random Forests
#                 for est in [100]:  # default = 10
#                     run(RandomForestClassifier(n_estimators=est, n_jobs=-1,
#                                                verbose=True, bootstrap=True),
#                         f'{model_name}_RF_{est}e', data_vectors)

                print(f'\n========== {model_name} cycle finished in {int(np.ceil(time.time() - cycle_time))} seconds. ==========\n\n\n')

                # Save model dataframes
                acc_df = acc_df.transpose()
                tim_df = tim_df.transpose()

                acc_df.to_csv(f'{data_out}/{model_name}_{now}_acc.csv')
                out_df.to_csv(f'{data_out}/{model_name}_{now}_out.csv')
                tim_df.to_csv(f'{data_out}/{model_name}_{now}_tim.csv')

Creating average feature vectors... 

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


Done in 48.93 seconds!
Iteration 1, loss = 0.45156193
Iteration 2, loss = 0.35701665
Iteration 3, loss = 0.35362293
Iteration 4, loss = 0.35177234
Iteration 5, loss = 0.35046596
Iteration 6, loss = 0.34915801
Iteration 7, loss = 0.34806799
Iteration 8, loss = 0.34694963
Iteration 9, loss = 0.34558745
Iteration 10, loss = 0.34431827
Iteration 11, loss = 0.34287248
Iteration 12, loss = 0.34177714
Iteration 13, loss = 0.34078799
Iteration 14, loss = 0.33999503
Iteration 15, loss = 0.33916011
Iteration 16, loss = 0.33860722
Iteration 17, loss = 0.33813154
Iteration 18, loss = 0.33776646
Iteration 19, loss = 0.33743718
Iteration 20, loss = 0.33718154
Iteration 21, loss = 0.33683930
Iteration 22, loss = 0.33650721
Iteration 23, loss = 0.33624096
Iteration 24, loss = 0.33592348
Iteration 25, loss = 0.33589395
Iteration 26, loss = 0.33573907
Iteration 27, loss = 0.33558695
Iteration 28, loss = 0.33513817
Iteration 29, loss = 0.33513090
Iteration 30, loss = 0.33483620
Iteration 31, loss = 0.334

Iteration 30, loss = 0.33364716
Iteration 31, loss = 0.33344206
Iteration 32, loss = 0.33320473
Iteration 33, loss = 0.33321087
Iteration 34, loss = 0.33276440
Iteration 35, loss = 0.33293821
Iteration 36, loss = 0.33274912
Iteration 37, loss = 0.33241259
Iteration 38, loss = 0.33236137
Iteration 39, loss = 0.33236151
Iteration 40, loss = 0.33217777
Iteration 41, loss = 0.33201409
Iteration 42, loss = 0.33171874
Iteration 43, loss = 0.33159587
Iteration 44, loss = 0.33153589
Iteration 45, loss = 0.33142328
Iteration 46, loss = 0.33148228
Iteration 47, loss = 0.33140096
Iteration 48, loss = 0.33126905
Iteration 49, loss = 0.33089288
Iteration 50, loss = 0.33089860
Iteration 51, loss = 0.33080551
Iteration 52, loss = 0.33066314
Iteration 53, loss = 0.33057230
Iteration 54, loss = 0.33051816
Iteration 55, loss = 0.33036146
Iteration 56, loss = 0.33028444
Iteration 57, loss = 0.33032823
Iteration 58, loss = 0.33009124
Iteration 59, loss = 0.32995585
Iteration 60, loss = 0.32992263
Iteratio

Iteration 121, loss = 0.31914481
Iteration 122, loss = 0.31904361
Iteration 123, loss = 0.31892922
Iteration 124, loss = 0.31874002
Iteration 125, loss = 0.31886229
Iteration 126, loss = 0.31868389
Iteration 127, loss = 0.31838475
Iteration 128, loss = 0.31824801
Iteration 129, loss = 0.31830077
Iteration 130, loss = 0.31805520
Iteration 131, loss = 0.31789417
Iteration 132, loss = 0.31795366
Iteration 133, loss = 0.31761064
Iteration 134, loss = 0.31777525
Iteration 135, loss = 0.31743182
Iteration 136, loss = 0.31731562
Iteration 137, loss = 0.31754506
Iteration 138, loss = 0.31715494
Iteration 139, loss = 0.31701604
Iteration 140, loss = 0.31696742
Iteration 141, loss = 0.31691778
Iteration 142, loss = 0.31663614
Iteration 143, loss = 0.31636704
Iteration 144, loss = 0.31660111
Iteration 145, loss = 0.31636360
Iteration 146, loss = 0.31618425
Iteration 147, loss = 0.31609970
Iteration 148, loss = 0.31605095
Iteration 149, loss = 0.31615570
Training loss did not improve more than tol

...

Iteration 4, loss = 0.32686018
Iteration 5, loss = 0.32380312
Iteration 6, loss = 0.32135167
Iteration 7, loss = 0.31929012
Iteration 8, loss = 0.31776977
Iteration 9, loss = 0.31640082
Iteration 10, loss = 0.31505255
Iteration 11, loss = 0.31418938
Iteration 12, loss = 0.31301485
Iteration 13, loss = 0.31192795
Iteration 14, loss = 0.31108391
Iteration 15, loss = 0.31026172
Iteration 16, loss = 0.30930577
Iteration 17, loss = 0.30837359
Iteration 18, loss = 0.30769654
Iteration 19, loss = 0.30735679
Iteration 20, loss = 0.30652781
Iteration 21, loss = 0.30585064
Iteration 22, loss = 0.30541696
Iteration 23, loss = 0.30468057
Iteration 24, loss = 0.30440909
Iteration 25, loss = 0.30385237
Iteration 26, loss = 0.30326209
Iteration 27, loss = 0.30305570
Iteration 28, loss = 0.30274703
Iteration 29, loss = 0.30186659
Iteration 30, loss = 0.30130178
Iteration 31, loss = 0.30127980
Iteration 32, loss = 0.30058886
Iteration 33, loss = 0.30042610
Iteration 34, loss = 0.29991416
Iteration 35, 

Iteration 11, loss = 0.31461378
Iteration 12, loss = 0.31388116
Iteration 13, loss = 0.31259574
Iteration 14, loss = 0.31163716
Iteration 15, loss = 0.31099295
Iteration 16, loss = 0.31009893
Iteration 17, loss = 0.30953503
Iteration 18, loss = 0.30889496
Iteration 19, loss = 0.30818061
Iteration 20, loss = 0.30737227
Iteration 21, loss = 0.30689407
Iteration 22, loss = 0.30645054
Iteration 23, loss = 0.30591376
Iteration 24, loss = 0.30540176
Iteration 25, loss = 0.30490384
Iteration 26, loss = 0.30439915
Iteration 27, loss = 0.30436437
Iteration 28, loss = 0.30379046
Iteration 29, loss = 0.30338572
Iteration 30, loss = 0.30322348
Iteration 31, loss = 0.30252770
Iteration 32, loss = 0.30200493
Iteration 33, loss = 0.30165453
Iteration 34, loss = 0.30131892
Iteration 35, loss = 0.30124615
Iteration 36, loss = 0.30064187
Iteration 37, loss = 0.30050367
Iteration 38, loss = 0.30031407
Iteration 39, loss = 0.29974724
Iteration 40, loss = 0.29949135
Iteration 41, loss = 0.29927645
Iteratio

Iteration 131, loss = 0.28234292
Iteration 132, loss = 0.28227600
Iteration 133, loss = 0.28194930
Iteration 134, loss = 0.28202393
Iteration 135, loss = 0.28211511
Iteration 136, loss = 0.28189162
Iteration 137, loss = 0.28157392
Iteration 138, loss = 0.28194283
Iteration 139, loss = 0.28138719
Iteration 140, loss = 0.28136084
Iteration 141, loss = 0.28129754
Iteration 142, loss = 0.28154922
Iteration 143, loss = 0.28081411
Iteration 144, loss = 0.28087181
Iteration 145, loss = 0.28102455
Iteration 146, loss = 0.28031509
Iteration 147, loss = 0.28062127
Iteration 148, loss = 0.28082564
Iteration 149, loss = 0.28094795
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.
100ft_10mw_5cx_lm_NN_1000iter_reluactiv_1.0pt_1e-05tl [ 6] | 87.995 % |  322 s
Iteration 1, loss = 0.38013708
Iteration 2, loss = 0.33803292
Iteration 3, loss = 0.33108068
Iteration 4, loss = 0.32718753
Iteration 5, loss = 0.32440224
Iteration 6, loss = 0.32249474
Iteration 7, loss

Iteration 13, loss = 0.31175610
Iteration 14, loss = 0.31106886
Iteration 15, loss = 0.30999456
Iteration 16, loss = 0.30890774
Iteration 17, loss = 0.30827728
Iteration 18, loss = 0.30748989
Iteration 19, loss = 0.30667399
Iteration 20, loss = 0.30627592
Iteration 21, loss = 0.30579680
Iteration 22, loss = 0.30523755
Iteration 23, loss = 0.30433304
Iteration 24, loss = 0.30394728
Iteration 25, loss = 0.30348415
Iteration 26, loss = 0.30292558
Iteration 27, loss = 0.30256055
Iteration 28, loss = 0.30208432
Iteration 29, loss = 0.30152335
Iteration 30, loss = 0.30116645
Iteration 31, loss = 0.30062891
Iteration 32, loss = 0.30032171
Iteration 33, loss = 0.29987969
Iteration 34, loss = 0.29962762
Iteration 35, loss = 0.29937588
Iteration 36, loss = 0.29872464
Iteration 37, loss = 0.29848471
Iteration 38, loss = 0.29821821
Iteration 39, loss = 0.29774310
Iteration 40, loss = 0.29768323
Iteration 41, loss = 0.29723786
Iteration 42, loss = 0.29686490
Iteration 43, loss = 0.29664314
Iteratio

Iteration 125, loss = 0.28183325
Iteration 126, loss = 0.28184627
Iteration 127, loss = 0.28167483
Iteration 128, loss = 0.28169133
Iteration 129, loss = 0.28136831
Iteration 130, loss = 0.28151177
Iteration 131, loss = 0.28120129
Iteration 132, loss = 0.28119302
Iteration 133, loss = 0.28094166
Iteration 134, loss = 0.28111400
Iteration 135, loss = 0.28060969
Iteration 136, loss = 0.28070659
Iteration 137, loss = 0.28067756
Iteration 138, loss = 0.28056488
Iteration 139, loss = 0.28037222
Iteration 140, loss = 0.28007174
Iteration 141, loss = 0.28042036
Iteration 142, loss = 0.28009129
Iteration 143, loss = 0.27983907
Iteration 144, loss = 0.27972186
Iteration 145, loss = 0.27957262
Iteration 146, loss = 0.27989853
Iteration 147, loss = 0.27947430
Iteration 148, loss = 0.27926203
Iteration 149, loss = 0.27919222
Iteration 150, loss = 0.27922517
Iteration 151, loss = 0.27939416
Iteration 152, loss = 0.27946240
Training loss did not improve more than tol=0.000010 for two consecutive epo